# Comp 551 Tutorial 2 : Scikit-learn

Oct 3, 2018

### Things we'll cover today

1) **What is scikit-learn and why should I care about it**

Machine Learning is a Machine Learning utility library. It contains standard models, evaluation metrics and more.

2) **How to go about doing ML (with scikit learn)**
  - ML as a single pipeline
    - Most common data preprocessing steps
      - train-test split
      - vectorization of textual features (only for textual features)
      - TF-IDF (only for textual features)
      - normalization
      - one-hot encoding of labels (for classification problems only)
    - Training
      - fit (closed form or gradient descent)
      - predict
    - Evaluation
      - metrics : measure accuracy - precision / recall / f-score
      - Cross validation
      - Grid Search

1) **What is scikit-learn and why should I care about it**

- Scikit-learn is a python-based free ML library that provides well-implemented off-the-shelf implementations for almost all ML operations.
- Implementing ML from scratch that is scalable, efficient, and error-free is really very very hard.

*Disclaimer* : Availability of off-the-shelf implementations of ML doesn't invalidates the need to know the algorithms details.

2) **How to go about doing ML (with scikit learn)**




2.1) **ML as a single pipeline**
- Almost all *scalable* ML models follows the style of development in a pipeline. It eases the pain of thinking through the complex ML processes.
- Keep this pipeline in mind while developing any ML model.

![alt text](http://cs.mcgill.ca/~sthaku3/other_media/COMP551_tutorial_2/Steps.png)


- P.S. : Closed-form solution seeking ML don't follow this suit.

From here on, we'll explain the concepts behind each of these steps with a real dataset called **News20group** as hosted [here](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). So, let's import it now. News articles (18 000 instances) belongning to 20 different classes.

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset='all')

In [2]:
type(newsgroups)

sklearn.utils.Bunch

In [3]:
# Bunch has data and target attributes
print(newsgroups.data)
print(newsgroups.target)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
# Print first 10 elements of data
list(newsgroups.data)[1:10]

['From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)\nSubject: Which high-performance VLB video card?\nSummary: Seek recommendations for VLB video card\nNntp-Posting-Host: midway.ecn.uoknor.edu\nOrganization: Engineering Computer Network, University of Oklahoma, Norman, OK, USA\nKeywords: orchid, stealth, vlb\nLines: 21\n\n  My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-performance VLB card\n\n\nPlease post or email.  Thank you!\n\n  - Matt\n\n-- \n    |  Matthew B. Lawson <------------> (mblawson@essex.ecn.uoknor.edu)  |   \n  --+-- "Now I, Nebuchadnezzar, praise and exalt and glorify the King  --+-- \n    |   of heaven, because everything he does is right and all his ways  |   \n    |   are just." - Nebuchadnezzar, king of Babylon, 562 B.C.           |   \n',
 'F

In [5]:
len(list(newsgroups.data))

18846

In [6]:
# This is how the target looks like
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

**2.1.1) Most common data-proprocessing steps**
![alt text](http://cs.mcgill.ca/~sthaku3/other_media/COMP551_tutorial_2/Preprocessing.png)

**2.1.1.1) Train-test split**

In [10]:
## Simple way to do split would be to use scikit-learn's `train_test_split` method
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newsgroups.data, newsgroups.target, train_size=0.8, test_size=0.2)
print(len(X_train)) 
train tesprint(len(y_train))
print(y_train[0])

SyntaxError: invalid syntax (<ipython-input-10-7654d00dacf5>, line 5)

In [ ]:
print(X_train[1])

**2.1.1.2) Vectorization of textual features (applicable only for textual features)**

A very simple approach to represent textual features such as documents or sentences as numerical value is to use each word as an atomic type and as a basis for a vector space. For example imagine a world where there exist only 3 words: “Apple”, “Orange”, and “Banana” and every sentence or document is made of them. They become the basis of a 3 dimensional vector space:

```
Apple  ==>> [1,0,0]
Banana ==>> [0,1,0]
Orange ==>> [0,0,1]
```

This representation is called **one_hot** as it is always a vector of zeros with 1 on the position of the word.

Then a “sentence” or a “document” is simply the linear combination of these vectors where the number of the counts of appearance of the words is the coefficient along that dimension. For example:

```
d3 = "Apple Orange Orange Apple" ==>> [2,0,2]
d4 = "Apple Banana Apple Banana" ==>> [2,2,0]
d1 = "Banana Apple Banana Banana Banana Apple" ==>> [2,4,0]
d2 = "Banana Orange Banana Banana Orange Banana" ==>> [0,4,2]
d5 = "Banana Apple Banana Banana Orange Banana" ==>> [1,4,1]
```
Two disadvantages:
- We are not capturing the ordering of the words (big drawback for sentimental analysis. 
- What is the vocabulary contains 150 000 different words, the array would be really big.

## Now we transform text into vectors



In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print(vectorizer)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [0]:
vectors_train = vectorizer.fit_transform(X_train)

# and we do the same for test data. remember to use the same vectorizer, only transform (why?), we don't want to build vocabulary (unique words) based on testset
vectors_test = vectorizer.transform(X_test)

In [0]:
print(vectorizer.vocabulary_)
print(len(vectorizer.vocabulary_))

{'from': 60031, 'mvalente': 89135, 'draco': 51463, 'lnec': 80398, 'pt': 101609, 'subject': 116984, 'on': 93897, 'dos': 51212, 'or': 94318, 'windows': 130603, 'organization': 94446, 'psgnet': 101496, 'portland': 99644, 'oregon': 94421, 'us': 125924, 'newsreader': 90983, 'tin': 121135, 'version': 127405, 'pl8': 98703, 'lines': 79955, '22': 9529, 'posting': 99764, 'this': 120607, 'request': 106023, 'again': 27093, 'since': 112831, 'the': 120290, 'last': 78533, 'one': 93921, 'had': 64729, 'no': 91679, 'title': 121222, 'looking': 80669, 'for': 59273, 'server': 111405, 'software': 114089, 've': 127120, 'already': 28152, 'seen': 110973, 'desqview': 48919, 'and': 28722, 'xvision': 133346, 'but': 37195, 'would': 131342, 'like': 79815, 'to': 121432, 'be': 33283, 'aware': 31678, 'of': 93420, 'other': 94772, 'solutions': 114207, 'namely': 89947, 'public': 101683, 'domain': 51026, 'shareware': 111798, 'implementations': 69913, 'also': 28160, 'know': 76953, 'in': 70068, 'commercial': 43083, 'case': 

In [0]:
print(vectors_test.shape)

(3770, 135716)


In [0]:
# This takes a lot of memory, therefore it can't be store as a numpy array, but a sparse matrix from scipy 
print(vectors_test[0])

scipy.sparse.csr.csr_matrix

**2.1.1.3) Tf–idf term weighting(only for textual features)**

In a large text corpus, some words are quite common (e.g. “the”, “a”, “is” in English). These words don't always convery meaningful information. However, if we were to feed the direct count data directly to a classifier those very frequent terms would shadow the frequencies of rarer yet more interesting terms.

In order to re-weight the count features into floating point values suitable for usage by a classifier it is very common to use the tf–idf transform.

tf means term-frequency while tf–idf means term-frequency times inverse document-frequency:

$$\text{tf-idf} = \text{tf}(t,d)\  \text{X} \ \text{idf}(t)$$ 
  where :
- $t$ =  term/token/word 
 
- $d$ = document or a paragraph

- The $\text{tf}(t,d)$ is the number of times a token $t$ appears in a document $d$

- The idf is calculted as using the following formula:
  $$log\frac{n_{d}}{1+ \text{df}(d,t)}$$
  - where
   - $n_{d}$ is the total number of documents
   - $\text{df}(d,t)$ is the number of documents $d$ that contain term $t$ 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
vectors_test_idf = vectorizer.transform(X_test)
print(X_train[1:2])

['From: drg@biomath.mda.uth.tmc.edu (David Gutierrez)\nSubject: Re: LCIII or used IIci - which should I get?\nOrganization: Univ. Texas M.D. Anderson Cancer Center\nLines: 15\nDistribution: world\nNNTP-Posting-Host: ratatosk.mda.uth.tmc.edu\n\nIn article <1993Apr24.232542.6070@cheshire.oxy.edu> erik@cheshire.oxy.edu\n(Erik Adams) writes:\n>I am, at long last, going to replace my beloved 512ke.\n>I am looking at a new LC III and a used IIci.  Prices\n>have yet to be worked out, so I\'m just thinking right now\n>about their merits and drawbacks.\n\nI\'d get the IIci. It\'s more expandable, just as fast, and preserves the\noption to run System 6.\n\nDavid Gutierrez\ndrg@biomath.mda.uth.tmc.edu\n\n"Only fools are positive." - Moe Howard\n\n']


In [0]:
print(vectors_train_idf.shape)
print(vectors_train_idf)

(15076, 135716)
  (0, 126728)	0.22046860654263312
  (0, 83382)	0.1496146438736416
  (0, 37368)	0.045698091305663456
  (0, 84576)	0.047608643711587045
  (0, 79323)	0.0756450392439267
  (0, 98881)	0.06709243165288829
  (0, 29981)	0.09597993889048523
  (0, 56281)	0.13004902957248435
  (0, 29263)	0.04672912422835623
  (0, 65538)	0.035186084377086806
  (0, 134240)	0.03579571470543099
  (0, 69230)	0.03856352455287698
  (0, 55414)	0.0751900400899243
  (0, 79643)	0.13815219870675996
  (0, 83068)	0.15229647946392166
  (0, 130595)	0.10374883214032503
  (0, 31516)	0.08508464553717922
  (0, 70224)	0.18837195382764946
  (0, 100807)	0.08641458017499297
  (0, 99727)	0.08231558029989948
  (0, 25719)	0.04580813934366583
  (0, 39655)	0.07601860076570917
  (0, 43083)	0.11707014661837335
  (0, 70068)	0.05406832599697996
  (0, 76953)	0.10282135187266846
  :	:
  (15075, 104382)	0.018577777034817253
  (15075, 53120)	0.038393417165550656
  (15075, 84576)	0.041907507340851094
  (15075, 65538)	0.020648366548190

**2.1.1.4) Normalization**::
While not mandatory, normalization usually improves the performance of the learner significantly. It ensures that the learner learns from the data on similar scales across features. There are many ways of normalizing the data.

For now we'll stick to the default *l2* provided by scikit-learn.

In [0]:
from sklearn.preprocessing import normalize

vectors_train_normalized = normalize(vectors_train)
vectors_test_normalized = normalize(vectors_test)

**2.1.1.5) One-hot encoding of labels (for classification problems only)**
The integer nature of the labels is not amenable for classification tasks. However, scikit-learn above internally handles the integer nature of our labels. In most other cases, the programmers need to represent them in a format that allows handling them explicitly. One such popular format is one-hot encoding. This one-hot encoding works similar to as explained in section 2.1.1.2. So, we are only refering to a function [here](http://scikit-learn.org/stable/modules/preprocessing_targets.html#) that does that for you but for labels in the context of classification.

![alt text](http://cs.mcgill.ca/~sthaku3/other_media/COMP551_tutorial_2/training.png)

In [0]:
## Now we instantiate the classifier. Remember this can be any classifier, even the one you make.
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

print(clf)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


**2.1.2.1) Fit**

In [0]:
## Scikit Learn API is very simple and straightforward, which contains the basic commands:
## `fit` to learn your parameters
clf.fit(vectors_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**2.1.2.2) Predict**

In [0]:
## We get the predicted class
print(vectors_test)
tic = time.time()
y_pred = clf.predict(vectors_test)
toc = time.time()
print("Time taken" : toc - tic)## So now we see we have a set of predictions. 
y_pred

  (0, 4152)	1
  (0, 4189)	1
  (0, 13018)	1
  (0, 13201)	1
  (0, 24126)	1
  (0, 27511)	2
  (0, 27743)	1
  (0, 28532)	1
  (0, 29941)	2
  (0, 30009)	2
  (0, 30109)	1
  (0, 30120)	1
  (0, 30448)	1
  (0, 30591)	7
  (0, 30912)	1
  (0, 31129)	1
  (0, 31133)	1
  (0, 31446)	1
  (0, 31528)	4
  (0, 31540)	1
  (0, 31594)	1
  (0, 31860)	2
  (0, 32124)	1
  (0, 34235)	1
  (0, 34372)	3
  :	:
  (3769, 76598)	1
  (3769, 76883)	3
  (3769, 78443)	2
  (3769, 79994)	2
  (3769, 83759)	1
  (3769, 88766)	1
  (3769, 89073)	2
  (3769, 99062)	1
  (3769, 102681)	1
  (3769, 105617)	1
  (3769, 105657)	1
  (3769, 109272)	1
  (3769, 110785)	1
  (3769, 110961)	1
  (3769, 122234)	1
  (3769, 123791)	1
  (3769, 124864)	1
  (3769, 124938)	1
  (3769, 125915)	1
  (3769, 126010)	1
  (3769, 126851)	2
  (3769, 133299)	1
  (3769, 135931)	1
  (3769, 136477)	1
  (3769, 136690)	1


array([ 2, 18,  8, ..., 12, 13,  5])

### Evaluation

To see how good or bad your classifier did, you should check the predictions with the gold standard dataset. The cool thing about scikit-learn is that it gives you several metrics to do so. You can even use your own classifier and use the list of predicted classes and gold standard classes to compare.

![alt text](http://cs.mcgill.ca/~sthaku3/other_media/COMP551_tutorial_2/metrics.png)

In [0]:
from sklearn import metrics

The `metrics` class provides a set of useful metrics you can use for your needs. For any classification task, you need to report mainly these metrics:

- Accuracy : (TP + TN) / (TP + TN + FP + FN)
- Precision : TP / (TP + FP)
- F1 Score : 2TP / (2TP + FP + FN)
- Recall Score : TP / (TP + FN)

Remember, when we do multi-class classification, we use it in `macro` average mode, where we calculate metrics for each label, and find their unweighted mean. You can also instead use other averaging modes such as `micro`, `weighted`, `samples`

In [0]:
metrics.accuracy_score(y_test, y_pred)

0.903448275862069

In [0]:
metrics.precision_score(y_test, y_pred, average='macro')

0.9021714314002607

In [0]:
metrics.f1_score(y_test, y_pred, average='macro')

0.9006473028048397

In [0]:
metrics.recall_score(y_test, y_pred, average='macro')

0.8999497390966233

In [0]:
## You can show all of this in a single call
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       155
           1       0.84      0.83      0.84       190
           2       0.85      0.85      0.85       184
           3       0.81      0.80      0.80       173
           4       0.86      0.88      0.87       206
           5       0.92      0.88      0.90       177
           6       0.87      0.88      0.87       202
           7       0.90      0.88      0.89       197
           8       0.97      0.97      0.97       177
           9       0.93      0.95      0.94       205
          10       0.94      0.97      0.95       201
          11       0.98      0.95      0.96       195
          12       0.84      0.87      0.85       184
          13       0.92      0.94      0.93       197
          14       0.95      0.97      0.96       215
          15       0.92      0.96      0.94       194
          16       0.89      0.92      0.90       192
          17       0.98    

In [0]:
### Cross Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf = MultinomialNB(alpha=.01)
scores = cross_val_score(clf, vectors_train, y_train, cv=5)
print(scores)

[0.87925901 0.89503311 0.8916501  0.88704319 0.8828619 ]


## Grid Search

When you are first searching for the best hyperparamters, its a good first strategy to run a grid search with the choice of hyperparameters to see which ones work the best for your dataset. 

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

tuned_parameters = [{'alpha': [0.01, 1, 0.5, 0.2, 0.1, 0.4], 'alpha1': [0.01, 1, 0.5, 0.2, 0.1, 0.4]}]

n_folds = 5

clf = MultinomialNB()
clf = GridSearchCV(clf, tuned_parameters, cv=n_folds, refit=False)
clf.fit(vectors_train, y_train)
scores = clf.cv_results_['mean_test_score']
scores_std = clf.cv_results_['std_test_score']
print('scores:',scores)
print('scores_std',scores_std)

scores: [0.87914566 0.83961263 0.86156806 0.87111966 0.87377288 0.86461926]
scores_std [0.00558718 0.00243207 0.00422124 0.00646701 0.00586275 0.00443983]


### Choice of Classifier
![Choosing the right estimator](http://scikit-learn.org/stable/_static/ml_map.png)

### References

1.[Scikit Learn](http://scikit-learn.org/)

### Useful Links

1. ROC Curve: http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
2. https://people.duke.edu/~ccc14/sta-663/BlackBoxOptimization.html